# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [4]:
# created table in workbench with below code

CREATE TABLE rentals_may (
 rental_id INT(11) UNIQUE NOT NULL,
 rental_date DATETIME,
 inventory_id INT(11) DEFAULT NULL,
 customer_id INT(11) DEFAULT NULL,
 return_date DATETIME,
 staff_id INT(11) DEFAULT NULL,
 last_update DATETIME,
 CONSTRAINT PRIMARY KEY (rental_id)
 );

SyntaxError: invalid syntax (<ipython-input-4-764e881a94e3>, line 3)

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [ ]:
# inserted data into new table with below query in workbench 

 INSERT INTO rentals_may
 SELECT *
 FROM rental
 WHERE MONTH(rental_date) = 05;

## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [ ]:
# used below query in workbench to create table 
    
CREATE TABLE rentals_june (
 rental_id INT(11) UNIQUE NOT NULL,
 rental_date DATETIME,
 inventory_id INT(11) DEFAULT NULL,
 customer_id INT(11) DEFAULT NULL,
 return_date DATETIME,
 staff_id INT(11) DEFAULT NULL,
 last_update DATETIME,
 CONSTRAINT PRIMARY KEY (rental_id)
 );

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [ ]:
# used below query in workbench to input data into table

 INSERT INTO rentals_june
 SELECT *
 FROM rental
 WHERE MONTH(rental_date) = 06;

## Check the number of rentals for each customer for May.


In [ ]:
 SELECT COUNT(*) AS number_of_rentals, customer_id
 FROM rentals_may
 GROUP BY customer_id
 ORDER BY COUNT(*) DESC;

## Check the number of rentals for each customer for June.
 

In [ ]:
 SELECT COUNT(*) AS number_of_rentals, customer_id
 FROM rentals_june
 GROUP BY customer_id
 ORDER BY COUNT(*) DESC;

## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [5]:
# import relevant packages first

import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

 ···········


In [6]:
# create connection with sakila database

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'

# create engine for connection

engine = create_engine(connection_string)

In [7]:
# save query as variable (first the query to find number of rentals for may)
query_1 = ('''SELECT MONTH(rental_date) AS month, COUNT(*) AS number_of_rentals, customer_id
 FROM rentals_may
 GROUP BY customer_id, MONTH(rental_date)
 ORDER BY COUNT(*) DESC; '''
 )

In [8]:
# save result of query into DF
rentals_may = pd.read_sql_query(query_1, engine)

In [9]:
# show DF
rentals_may

,month,number_of_rentals,customer_id
0,5,8,197
1,5,7,506
2,5,7,109
3,5,6,269
4,5,6,239
...,...,...,...
515,5,1,431
516,5,1,351
517,5,1,10
518,5,1,136


In [10]:
# second query 

query_2 = ('''SELECT MONTH(rental_date) AS month, COUNT(*) AS number_of_rentals, customer_id
 FROM rentals_june
 GROUP BY customer_id, MONTH(rental_date)
 ORDER BY COUNT(*) DESC; '''
 )

In [11]:
# save result of query into DF
rentals_june = pd.read_sql_query(query_2, engine)

In [12]:
# show second query as DF
rentals_june.head()

,month,number_of_rentals,customer_id
0,6,11,31
1,6,10,454
2,6,9,329
3,6,9,295
4,6,9,561


## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [13]:
rentals_may.shape

(520, 3)

In [14]:
rentals_june.shape

(590, 3)

In [15]:
# merged dataframes on customer_id so only the customers with rentals in both months remain

df_merge = pd.merge(rentals_may, rentals_june, on='customer_id')


In [16]:
df_merge

,month_x,number_of_rentals_x,customer_id,month_y,number_of_rentals_y
0,5,8,197,6,8
1,5,7,506,6,5
2,5,7,109,6,5
3,5,6,269,6,3
4,5,6,239,6,5
...,...,...,...,...,...
507,5,1,234,6,8
508,5,1,431,6,7
509,5,1,351,6,3
510,5,1,10,6,5


In [21]:
# changed columns names
df_merge.rename(columns = {'month_x' : 'may', 'number_of_rentals_x' : 'rentals_may', 'month_y' : 'june', 'number_of_rentals_y' : 'rentals_june'}, inplace = True)


In [25]:
df_merge

,may,rentals_may,customer_id,june,rentals_june
0,5,8,197,6,8
1,5,7,506,6,5
2,5,7,109,6,5
3,5,6,269,6,3
4,5,6,239,6,5
...,...,...,...,...,...
507,5,1,234,6,8
508,5,1,431,6,7
509,5,1,351,6,3
510,5,1,10,6,5


In [38]:
# this function checks which month had more rentals
def check_rental(may, june):
    if may > june:
        return 'more rentals in may'
    if june > may:
        return 'more rentals in june'
    else:
        return 'same amount in both months'
    

In [39]:
# apply function to merged df (I only got this through googling... hope its what the question meant!)
df_merge['check'] = df_merge.apply(lambda row : check_rental(row['rentals_may'],
                     row['rentals_june']), axis = 1)

In [40]:
# show df with new column which shows which month had more rentals
df_merge

,may,rentals_may,customer_id,june,rentals_june,check
0,5,8,197,6,8,same amount in both months
1,5,7,506,6,5,more rentals in may
2,5,7,109,6,5,more rentals in may
3,5,6,269,6,3,more rentals in may
4,5,6,239,6,5,more rentals in may
...,...,...,...,...,...,...
507,5,1,234,6,8,more rentals in june
508,5,1,431,6,7,more rentals in june
509,5,1,351,6,3,more rentals in june
510,5,1,10,6,5,more rentals in june
